<a href="https://colab.research.google.com/github/steven1174/Coursera_Capstone/blob/main/Toronto's%20Neighbours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toronto: Segmenting and Clustering Neighborhoods

## Libraries and Modules used

In [9]:
!pip install bs4
!pip install geopy

In [10]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

import requests
import folium
import pandas as pd
import numpy as np 

## Table of Boroughs

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

data = requests.get(url).text
soup = BeautifulSoup(data,'html5lib')

In [12]:
table = []

for row in soup.find_all('table')[0].find_all('td'):
    
    cell = {}
    if row.span.string == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = (row.p.b.string).strip()
        cell['Borough'] = ((row.span.text).split('(')[0]).strip()
        cell['Neighborhood'] = ((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).strip()
        table.append(cell)

In [13]:
table = pd.DataFrame(table)
table.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills)North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In order to be sure that all postal codes are unique, that is, they are not repeated on the created table, the number of rows of the table should be equal to the number of unique ***postal codes***. This is verified in the next line of code.

In [14]:
print(table['PostalCode'].shape[0] == table['PostalCode'].unique().shape[0])
print(table['PostalCode'].shape[0])

True
103


## Latitude and Longitud of Toronto's Postal Codes

In [15]:
data = pd.read_csv('https://raw.githubusercontent.com/steven1174/Coursera_Capstone/main/Geospatial_Coordinates.csv')
print('The number of Postal Codes are {}.'.format(data.shape[0]),'\n\n',data.head())

The number of Postal Codes are 103. 

   Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [16]:
final_table = pd.merge(table,data,left_on='PostalCode',right_on='Postal Code').drop('Postal Code',axis = 1)
final_table.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills)North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Toronto's maps and its neighborhoods

In [17]:
city = 'Toronto, Ontario'

location = Nominatim(user_agent="Toronto_explorer").geocode(city)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
print('Toronto has {} boroughs.'.format(len(final_table['Borough'].unique())))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.
Toronto has 15 boroughs.


In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(final_table['Latitude'], final_table['Longitude'], final_table['Borough'], final_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
#@title

CLIENT_ID = 'T1FT5O1QY1XIZTGGJB1WBH1DEK1GHWF12ACVV1ERWVJIRR5P' # your Foursquare ID
CLIENT_SECRET = 'ILKDYIFTUUYFPIDI32HC0SSD0HJVRDVT5OSFJUX3R0EWRKGM' # your Foursquare Secret
ACCESS_TOKEN = 'R3HCRHH5IER5UN1WKOIY22S0CN5SQUNHB3OZJCBSEHQMKQFR' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100